#### Import required libraries

In [1]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chat_models import init_chat_model
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from tqdm import tqdm

c:\python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Load PDF file

In [2]:
# Load documents
loader = PyPDFLoader(
    "../docs/courses_offered.pdf",
    mode="page")

docs = loader.load()
print('Loaded document count :', len(docs))

Loaded document count : 15


#### Split document into chunks of 500 chars 

In [3]:
# Split documents into chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = text_splitter.split_documents(docs)
print("Number of chunks created: ", len(chunks))

Number of chunks created:  34


#### Get Google Embedding Model

In [4]:
# Get embeddings model
embeddings_model = GoogleGenerativeAIEmbeddings(
    model="gemini-embedding-001")

#### Create Vector DB or load it from disk, if it is existing

In [5]:
# check whether vector store is present, load it if present otherwise create it
folder_path = "./courses_vectors"
if os.path.exists(folder_path):
    db = FAISS.load_local(folder_path, embeddings_model,
                          allow_dangerous_deserialization=True)
    print("Loaded FAISS index")
else:
    db = FAISS.from_documents(chunks, embeddings_model)
    print('Created FAISS index')
    db.save_local(folder_path)

Created FAISS index


#### Retrieve Matching Documents

In [6]:
question = "What is the course fee of C Language course"
retriever = db.as_retriever()
results = retriever.invoke(question)
print("Number of documents retrieved: ", len(results))

Number of documents retrieved:  4


#### Create Context with matching documents

In [7]:
# Create context
context = "\n".join([doc.page_content for doc in results])

#### Create prompt with query and context

In [8]:
prompt = f"""
Give me answer to my question based on the context.
{context}
Question: {question}
"""

#### Send Prompt to LLM and get response

In [9]:
# Access LLM with context and query
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

result = llm.invoke(prompt)
print(result.content)

The course fee for the C Language course is Rs. 4500/- (Includes printed course material).
